<a href="https://colab.research.google.com/github/deborahmasibo/Hamoye/blob/stage_b/Predicting_Energy_Efficiency_of_Buildings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from google.colab import drive

# Loading Data

In [2]:
# Mounting drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Accessing required working directory
os.chdir('/content/drive/My Drive/Hamoye/Stage B/Quiz')
# Loading dataset and parsing dates
energy = pd.read_csv('energydata_complete.csv', parse_dates = ['date'])

# Data Understanding

## preview

In [4]:
# 1st 5 rows
energy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [5]:
# Last 5 rows
energy.tail()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
19730,2016-05-27 17:20:00,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.7,...,23.2,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
19731,2016-05-27 17:30:00,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.7,...,23.2,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
19732,2016-05-27 17:40:00,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.7,...,23.2,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
19733,2016-05-27 17:50:00,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.7,...,23.2,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784
19734,2016-05-27 18:00:00,430,10,25.500000,46.600000,25.264286,42.971429,26.823333,41.156667,24.7,...,23.2,46.8450,22.200000,755.2,57.000000,4.000000,27.000000,13.200000,34.118851,34.118851


## Information

In [6]:
# Dataset datatypes
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         19735 non-null  datetime64[ns]
 1   Appliances   19735 non-null  int64         
 2   lights       19735 non-null  int64         
 3   T1           19735 non-null  float64       
 4   RH_1         19735 non-null  float64       
 5   T2           19735 non-null  float64       
 6   RH_2         19735 non-null  float64       
 7   T3           19735 non-null  float64       
 8   RH_3         19735 non-null  float64       
 9   T4           19735 non-null  float64       
 10  RH_4         19735 non-null  float64       
 11  T5           19735 non-null  float64       
 12  RH_5         19735 non-null  float64       
 13  T6           19735 non-null  float64       
 14  RH_6         19735 non-null  float64       
 15  T7           19735 non-null  float64       
 16  RH_7

## Description

In [63]:
# Dataset description
energy.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


# Data Preperation

## Missing Data

In [8]:
# Checking for missing data
energy.isnull().any().any()

False

There are no missing values.

## Duplicates

In [9]:
# Checking for duplicates
energy.duplicated().any().any()

False

There are no duplicates in the data.

## Outliers

In [10]:
# Checking for outliers

# Outliers function
def outliers(data):
  # IQR
  Q1, Q3, IQR = 0, 0, 0
  outliers = pd.DataFrame()
  # Numerical columns
  numerical = data.select_dtypes(include = ['int64', 'float64'])
  Q1 = numerical.quantile(0.25)
  Q3 = numerical.quantile(0.75)
  IQR = Q3 - Q1
  # Outliers
  outliers = numerical[((numerical < (Q1 - 1.5 * IQR)) |(numerical > (Q3 + 1.5 * IQR))).any(axis=1)]
  print(f'Number of outliers = {outliers.shape[0]}')
  print(f'Percentage = {(outliers.shape[0]/data.shape[0])*100}%')

In [11]:
# Function call
outliers(energy)

Number of outliers = 9250
Percentage = 46.87104129718774%


There are outliers present in the data.

# Tag Along Code

## Question 12
From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two d.p.?

In [12]:
# Features and label
X12 = energy['T2'].values.reshape(-1,1)
y12 = energy['T6'].values.reshape(-1,1)

In [16]:
# Train and test sets -> 80/20%
X12_train, X12_test, y12_train, y12_test = train_test_split(X12, y12, test_size = 0.2, 
                                                        random_state = 2)

Splitting the dataset to test model performance on unseen data.

In [17]:
# Linear Regression Model
linear12 = LinearRegression()
linear12.fit(X12_train, y12_train)
# Predictions
y_linear_pred_12 = linear12.predict(X12_test)

In [20]:
# R2_Score
r2_12 = r2_score(y12_test, y_linear_pred_12).round(2)
r2_12

0.64

## Question 13
Normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set. Answer the following questions:

What is the Mean Absolute Error (in two decimal places)?

In [21]:
# Dataset columns
energy.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [35]:
# Dropping specified columns
minmax = energy.drop(['date', 'lights'], axis = 1)
# Min-Max Scaling
scaler = MinMaxScaler()
minmax_scaled = scaler.fit_transform(minmax)
# Label
y13 = minmax_scaled[:,0]
# Features
X13 = minmax_scaled[:,1:]
# Train-test-split
X13_train, X13_test, y13_train, y13_test = train_test_split(X13, y13, test_size = 0.3, 
                                                        random_state = 42)


In [36]:
# Linear regression model
linear13 = LinearRegression()
linear13.fit(X13_train, y13_train)
# Predictions
y_linear_pred_13 = linear13.predict(X13_test)
# Mean Absolute Error
mae_13 = mean_absolute_error(y13_test, y_linear_pred_13).round(2)
mae_13

0.05

## Question 14
What is the Residual Sum of Squares (in two decimal places)

In [37]:
# RSS
rss_13 = np.sum(np.square(y13_test - y_linear_pred_13)).round(2)
rss_13

45.35

## Question 15
What is the Root Mean Squared Error (in three decimal places)?

In [38]:
# RMSE
rmse_13 = mean_squared_error(y13_test, y_linear_pred_13, squared = False).round(3)
rmse_13

0.088

## Question 16
What is the Coefficient of Determination (in two decimal places)?

In [39]:
# R2_Score
r2_13 = r2_score(y13_test, y_linear_pred_13).round(2)
r2_13

0.15

## Question 17
Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [50]:
# Weights dataframe function
def weights(model, data):
  weights = pd.Series(model.coef_, data.columns).sort_values()
  coeff_df = pd.DataFrame(weights).reset_index()
  coeff_df.columns = [ 'Feature' , 'Weight']
  return coeff_df

In [51]:
# Linear regression model weights
result17 = weights(linear13, minmax.iloc[:, 1:])
result17

,Feature,Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


## Question 18
Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [52]:
# Ridge regression model
ridge18 = Ridge(alpha=0.4)
ridge18.fit(X13_train, y13_train)
# Predictions
y_ridge_pred_18 = ridge18.predict(X13_test)

In [53]:
# RMSE
rmse_18 = mean_squared_error(y13_test, y_ridge_pred_18,
                             squared = False).round(2)
rmse_18

0.09

## Question 19
Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [54]:
# Lasso Regression
lasso19 = Lasso(alpha=0.001)
lasso19.fit(X13_train, y13_train)
y_lasso_pred_19 = lasso19.predict(X13_test)

In [55]:
# Lasso regression model weights
result19 = weights(lasso19, minmax.iloc[:, 1:])
result19

,Feature,Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [59]:
# Number of features with non-zero weights
len(result19[result19['Weight'] != 0])

4

## Question 20
What is the new RMSE with the lasso regression? (Answer should be in three (3) decimal places)

In [62]:
# RMSE
rmse_20 = mean_squared_error(y13_test, y_lasso_pred_19, squared = False).round(3)
rmse_20

0.094